#### 저장된 모델 활용
- 모델 파일 종류
    * 가중치 및 절편 저장 파일 => 동일한 구조 모델 인스턴스 생성 후 사용 가능
    * 모델 전체 저장 파일 => 바로 로딩 후 사용 가능


[1] 모듈로딩 <hr>

In [6]:
# 모듈 로딩
import torch
import torch.nn as nn
# 모듈 로딩
# 모델관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 전처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [7]:
class IrisMCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4, 10)
        self.hidden_layer=nn.Linear(10, 5)
        self.out_layer=nn.Linear(5, 3)
    
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y=self.in_layer(input_data) # f1w1+f2w2+f3w3+b 요런 식이 10개(숫자10개)
        y=F.relu(y)                   # 범위 0이상
        
        # 은닉층 : 10개의 숫자 받아오기
        y=self.hidden_layer(y)
        y=F.relu(y)

        # 출력층 : 5개의 숫자 값 => 다중 분류 : 손실함수 CrossEntropyLoss가 내부에서 SoftMax를 처리해줌 (케라스나, 텐서플로우 쓸땐 써야함)
        return self.out_layer(y)

In [8]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [9]:

## 모델 파일 관련

### models 폴더 아래 프로젝트 폴더 아래 모델 파일을 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/MCF/'

# 저장 파일명
SAVE_FILE = SAVE_PATH + 'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL=SAVE_PATH+'model_all.pth'

[2] 모델 로딩 - 모델 전체 파일 사용

In [10]:
irisModel = torch.load(SAVE_MODEL, weights_only=False)

summary(irisModel)

Layer (type:depth-idx)                   Param #
IrisMCFModel                             --
├─Linear: 1-1                            50
├─Linear: 1-2                            55
├─Linear: 1-3                            18
Total params: 123
Trainable params: 123
Non-trainable params: 0

[3] -예측 수행 <hr>


In [11]:
data=[float(x) for x in input("SL, SW, PL, PW : ").split(',')]


In [12]:
dataTS=torch.FloatTensor(data).reshape(1,-1)
irisModel

IrisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)

In [14]:
#함수화
irisModel.eval()
with torch.no_grad():
    # 추론 평가
    pre_val = irisModel(dataTS)

pre_val

tensor([[ 4.5771,  1.7690, -4.9159]])

In [15]:
def predictReg(data):
    irisModel = torch.load(SAVE_MODEL, weights_only=False)

    #함수화
    irisModel.eval()
    with torch.no_grad():
    # 추론 평가
        pre_val = irisModel(dataTS)

    return pre_val
